# ***3. Đóng gói hoàn chỉnh 2 hàm: Detect cắt ảnh vs nhận diện ảnh + THỰC HIỆN LƯU ảnh cắt vào cropped_imgs và lưu output vào .csv***

## Lồng hai đoạn code Python trên sao cho chúng có thể thực hiện một cách liên tục, từ việc phát hiện và cắt biển đường đến nhận diện nội dung trên biển số, bạn có thể tiến hành như sau:

1. Hoàn chỉnh def: Phát hiện biển tên đường, sử dụng YOLO.
2. Hoàn chỉnh def: Nhận diện văn bản, dùng PaddleOCR. (Bonus thêm trường hợp confidence nhỏ, PaddleOCR ko nhận diện được)
3. Trong vòng lặp của quá trình PHÁT HIỆN BIỂN TÊN ĐƯỜNG: Gọi hàm NHẬN DIỆN VĂN BẢN để XỬ LÝ VĂN BẢN TỪNG ẢNH ĐÃ CẮT.



In [13]:
# Import necessary libraries
import csv
import cv2
import os
from IPython.display import display, Image
from yolov8 import Yolov8           # đặt yolov8.py ở cùng cấp bậc với file 
from paddleocr import PaddleOCR


In [14]:

# OCR from image. 
# Khi detect và cắt xong biển, gọi hàm này nhận diện từng chữ trên biển

def ocr_from_image(img_path):
    ocr = PaddleOCR(use_angle_cls=True, lang='vi')  # Initialize PaddleOCR with Vietnamese language
    results = ocr.ocr(img_path, cls=True)
    text_recog = []
    total_confidence = 0
    count = 0
    # for line in results:
    #     for result in line:
    #         text = result[1][0]
    #         confidence = result[1][1]
    #         text_recog.append(text)
    #         total_confidence += confidence
    #         count += 1

    # UPDATE để xử lý trường hợp biển có độ tin cậy thấp OCR ko xử lý được
    # Check if results is not None and is iterable
    if results:
        # Process results
        for line in results:
            # Add a check to make sure 'line' is not None
            if line:
                for result in line:
                    text = result[1][0]
                    confidence = result[1][1]
                    text_recog.append(text)
                    total_confidence += confidence
                    count += 1
            else:
                print("A line within the results is None.")
    else:
        print("No text detected in the image or OCR failed to return results.")

    text = ' '.join(text_recog)
    average_confidence = total_confidence / count if count else 0
    return text, round(average_confidence, 2)


In [15]:
# code tổng hợp: Detect cắt frame and recognition tên trên biển

# Set up the YOLO model ở ngoài def thay vì gọi model_path mỗi lần khi duyệt từng ảnh trong 1 folder
model_path = r"model\road.onnx"
# Set up the Yolo model
yolo_model = Yolov8()
yolo_model.set_up_model(model_path)
yolo_model.model.warmup(imgsz=(1 , 3, *yolo_model.imgsz))

# Function for object detection and cropping
def detect_and_ocr(yolo_model, image_path):
    # Read the image or video frame
    cap = cv2.VideoCapture(image_path)
    ret, frame = cap.read()
    all_recognized_text_list = []
    # If a frame is obtained, perform detection and show results
    if ret:
        results = yolo_model.inference(frame)
        for result in results:
            x1, y1, x2, y2, conf, cls = result    # confidence, class label cls

            # Hiển thị rectangle và putText
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.putText(frame, str(yolo_model.names[int(cls)]), (int(x1), int(y1)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            # Cropped image used for OCR
            cropped_frame = frame[int(y1):int(y2), int(x1):int(x2)]
            # cropped_filename = "cropped_image_{}.jpg".format(idx)   # đặt tên theo index
            cropped_filename = "cropped_image_{:.4f}.jpg".format(conf)    # tên ảnh đính kèm với confidence
            # cv2.imwrite(cropped_filename, cropped_frame) 
            # Lưu ảnh vào thư mục chứ ko để lưu bên ngoài nữa 
            # Tạo đường dẫn file đầy đủ
            cropped_filename = cropped_folder_path + "\\" + cropped_filename
            cv2.imwrite(cropped_filename, cropped_frame)
     
            print(f"Cropped {cropped_filename} img")

            # Perform OCR on the image. OCR NGAY TRÊN ẢNH VỪA CẮT
            recognized_text, avg_confidence = ocr_from_image(cropped_filename)

            # print để xem từng frame được cắt
            print(recognized_text, avg_confidence)
            # Lưu vào all-recognized_text đẻ return cho tiện
            all_recognized_text_list.append(recognized_text)

        # frame = cv2.resize(frame, (640, 640))   # đồng nhất output ở size này (có thể ảnh hưởng đến quality img)
        print(f" SUCCESSFUL CROPPED {image_path} img")
        return all_recognized_text_list      # [',', 'Le Dai Hanh'], hoặc ['', '']
    else:
        print("Failed to read from image_pathpathpath")


Ultralytics YOLOv8.0.207  Python-3.11.5 torch-2.1.1+cpu CPU (Intel Core(TM) i3-4030U 1.90GHz)
Loading model\road.onnx for ONNX Runtime inference...


## Input folder: 
Test_data folder, full_data folder, lưu output vào file .csv

(BUG SIÊU TO, VÌ CHƯA BẬT ĐỒNG BÔ HÓA ĐÁM MÂY NÊN LÀ KHI LOAD: 
D:\OneDrive - codemely\Data_MiniProject\Nhận diện biển báo\img\Aug_use_Roboflow\332_original_img
thì gặp lỗi => PHẢI CHECK LẠI TỪ ĐẦU ĐẾN CUỐI file .ipynb)


In [4]:
# Import necessary libraries
import csv
import cv2
import os
from IPython.display import display, Image
from yolov8 import Yolov8
from paddleocr import PaddleOCR


In [5]:
model_path = r"model\road.onnx"

# Set up the Yolo model ở ngoài thay vì gọi model_path mỗi lần duyệt 1 ảnh của folder
yolo_model = Yolov8()
yolo_model.set_up_model(model_path)
yolo_model.model.warmup(imgsz=(1 , 3, *yolo_model.imgsz))


Ultralytics YOLOv8.0.207  Python-3.11.5 torch-2.1.1+cpu CPU (Intel Core(TM) i3-4030U 1.90GHz)
Loading model\road.onnx for ONNX Runtime inference...


In [16]:
'''
Tắt: Dòng thời gian và các thông điệp log
[2023/11/09 06:14:19] ppocr DEBUG: dt_boxes num : 3, elapsed : 0.17255139350891113
[2023/11/09 06:14:19] ppocr DEBUG: cls num  : 3, elapsed : 0.14942288398742676
[2023/11/09 06:14:20] ppocr DEBUG: rec_res num  : 3, elapsed : 0.9604041576385498
'''
import logging

# Cấu hình root logger
logging.basicConfig(level=logging.WARNING) # chỉ những thông điệp có mức độ quan trọng từ WARNING trở lên mới được hiển thị, và các thông điệp DEBUG hoặc INFO sẽ không được in ra.
                                    # Hãy chắc chắn rằng bạn không đang gọi basicConfig ở bất kỳ nơi nào khác trong mã của bạn với một level thấp hơn, vì cài đặt logger chỉ có thể thực hiện một lần và lần cài đặt đầu tiên sẽ có hiệu lực.
# Nếu bạn biết tên của logger mà thư viện OCR sử dụng, bạn có thể cấu hình trực tiếp:
logger = logging.getLogger('ppocr')
logger.setLevel(logging.WARNING)


In [17]:

# Load test folder
image_folder = r"D:\OneDrive - codemely\Proj1_NhanDienBienBaoTenDuong_CODEMELY\mecode\ResearchModel\DATA_TestResearch\sign_test_imgs"
# image_folder = r'D:\OneDrive - codemely\Proj1_NhậnDien\code\code\roadapp\roadapp\full_data\ocr_datatest'
output_csv = r'D:\OneDrive - codemely\Proj1_NhanDienBienBaoTenDuong_CODEMELY\mecode\ResearchModel\DATA_TestResearch\TEST_paddle_results.csv' # file CSV lưu kết quả
cropped_folder_path = r'D:\OneDrive - codemely\Proj1_NhanDienBienBaoTenDuong_CODEMELY\mecode\ResearchModel\DATA_TestResearch\crop_sign_test_imgs\\'


In [8]:
# Load 330 imgs
image_folder = r"D:\OneDrive - codemely\Proj1_NhậnDien\code\code\roadapp\roadapp\full_data\ocr_data_test"
# image_folder = r'D:\OneDrive - codemely\Proj1_NhậnDien\code\code\roadapp\roadapp\full_data\ocr_datatest'
output_csv = r"D:\OneDrive - codemely\Proj1_NhanDienBienBaoTenDuong_CODEMELY\code\roadapp\full_data\ocr_data_test\ocr_data_testocr_results.csv" # file CSV lưu kết quả
cropped_folder_path = r'D:\OneDrive - codemely\Proj1_NhậnDien\code\code\roadapp\roadapp\full_data\ocr_data_test\cropped_imgs\\'


In [18]:
# Mở file CSV để ghi
with open(output_csv, mode='a', newline='', encoding='utf-8') as file:  # Chú ý xem file có đang mở ko, đang mở ko thao tác được
    writer = csv.writer(file) 

    # Ghi tiêu đề cho cột trong CSV
    writer.writerow(['Image Name', 'OCR Result'])

    # Duyệt qua tất cả các file trong thư mục
    for image_name in os.listdir(image_folder):
        
        if image_name.lower().endswith(('.png', '.jpg')):
            print(f"START DETECT&REG {image_name} img")

            # Đường dẫn đầy tủ image_path, prepare for detect_and_ocr def()
            image_path = os.path.join(image_folder, image_name)

            # Thực hiện DETECT FRAME - OCR FRAME: tham số yolo_model và image_path
            all_recognized_text_list = detect_and_ocr(yolo_model, image_path)

            # Kiểm tra xem all_recognized_text_list có phải là một iterable không trước khi join
            if isinstance(all_recognized_text_list, list):
                all_recognized_text_str = ', '.join(all_recognized_text_list)
            else:
                # Xử lý trường hợp all_recognized_text_list không phải là một list
                # Có thể là ghi log, ném exception, hoặc một hành động thay thế nào đó
                print("all_recognized_text_list không phải là một list.")
                all_recognized_text_str = ''  # Hoặc một giá trị thay thế nào đó

            # Ghi kết quả vào file CSV
            writer.writerow([image_name, all_recognized_text_str])

            print(f"Save successfully DETECT&RECOG of {image_name} into .csv : {image_name, all_recognized_text_str}")
            print("-----------------------------------------------")

print(f"All images processed. Results saved to {output_csv}")


START DETECT&REG s1.jpg img
inference_time:  59 ms


RuntimeError: Couldn't load custom C++ ops. This can happen if your PyTorch and torchvision versions are incompatible, or if you had errors while compiling torchvision from source. For further information on the compatible versions, check https://github.com/pytorch/vision#installation for the compatibility matrix. Please check your PyTorch version with torch.__version__ and your torchvision version with torchvision.__version__ and verify if they are compatible, and if not please reinstall torchvision so that it matches your PyTorch install.

## Lần Test1: Full data được cắt - 

In [19]:
# Mở file CSV để ghi
with open(output_csv, mode='a', newline='', encoding='utf-8') as file:  # Chú ý xem file có đang mở ko, đang mở ko thao tác được
    writer = csv.writer(file) 

    # Ghi tiêu đề cho cột trong CSV
    writer.writerow(['Image Name', 'OCR Result'])

    # Duyệt qua tất cả các file trong thư mục
    for image_name in os.listdir(image_folder):
        
        if image_name.lower().endswith(('.png', '.jpg')):
            print(f"START DETECT&REG {image_name} img")

            # Đường dẫn đầy tủ image_path, prepare for detect_and_ocr def()
            image_path = os.path.join(image_folder, image_name)

            # Thực hiện DETECT FRAME - OCR FRAME: tham số yolo_model và image_path
            all_recognized_text_list = detect_and_ocr(yolo_model, image_path)

            # Kiểm tra xem all_recognized_text_list có phải là một iterable không trước khi join
            if isinstance(all_recognized_text_list, list):
                all_recognized_text_str = ', '.join(all_recognized_text_list)
            else:
                # Xử lý trường hợp all_recognized_text_list không phải là một list
                # Có thể là ghi log, ném exception, hoặc một hành động thay thế nào đó
                print("all_recognized_text_list không phải là một list.")
                all_recognized_text_str = ''  # Hoặc một giá trị thay thế nào đó

            # Ghi kết quả vào file CSV
            writer.writerow([image_name, all_recognized_text_str])

            print(f"Save successfully DETECT&RECOG of {image_name} into .csv : {image_name, all_recognized_text_str}")
            print("-----------------------------------------------")

print(f"All images processed. Results saved to {output_csv}")


START DETECT&REG cropped_image_0.3089.jpg img
inference_time:  120 ms


RuntimeError: Couldn't load custom C++ ops. This can happen if your PyTorch and torchvision versions are incompatible, or if you had errors while compiling torchvision from source. For further information on the compatible versions, check https://github.com/pytorch/vision#installation for the compatibility matrix. Please check your PyTorch version with torch.__version__ and your torchvision version with torchvision.__version__ and verify if they are compatible, and if not please reinstall torchvision so that it matches your PyTorch install.

NameError: name 'os' is not defined

In [ ]:
'''
Bug: ---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
d:\OneDrive - codemely\Proj1_NhậnDien\code\code\roadapp\roadapp\main_Cuong.ipynb Cell 27 line 1
     16 # Thực hiện DETECT FRAME - OCR FRAME: tham số model_path và image_path
     17 all_recognized_text_list = detect_and_ocr(model_path, image_path)
---> 18 all_recognized_text_str = ', '.join(all_recognized_text_list)
     20 # Ghi kết quả vào file CSV
     21 writer.writerow([image_name, all_recognized_text_str])

TypeError: can only join an iterable
all_recognized_text_list = set()
'''

# Kiểm tra xem all_recognized_text_list có phải là một iterable không trước khi join
if isinstance(all_recognized_text_list, list):
    all_recognized_text_str = ', '.join(all_recognized_text_list)
else:
    # Xử lý trường hợp all_recognized_text_list không phải là một list
    # Có thể là ghi log, ném exception, hoặc một hành động thay thế nào đó
    print("all_recognized_text_list không phải là một list.")
    all_recognized_text_str = ''  # Hoặc một giá trị thay thế nào đó



Finished after 50 min. 

# Load result_DetectYolov8&RecogPaddleocr & file đánh tay from Chu Anh Duc



In [ ]:

import pandas as pd

labeled_imgs_path = r"D:\OneDrive - codemely\Proj1_NhanDienBienBaoTenDuong_CODEMELY\code\roadapp\full_data\ocr_data_test\labeled_imgs.csv"
DetectRecog_result_path = r"D:\OneDrive - codemely\Proj1_NhanDienBienBaoTenDuong_CODEMELY\code\roadapp\full_data\ocr_data_test\ocr_data_testocr_results.csv"

df1 = pd.read_csv(labeled_imgs_path)
# Since we know the second file read correctly the first time, we'll keep it as is
df2 = pd.read_csv(DetectRecog_result_path)
data_file_2_full_renamed = df2.rename(columns={'Image Name': 'images_name'})


In [ ]:

# Merge the data on the 'images_name' column
# merged_data = pd.merge(df1, data_file_2_full_renamed, on='images_name', how='inner')   # how = 'inner' chỉ lấy cái cùng cột 
merged_data = pd.merge(df1, data_file_2_full_renamed, on='images_name', how='outer')   # bảo tồn tất cả các dòng từ cả hai DataFrame, ngay cả khi không có khớp đối ứng trong cột được chỉ định để hợp nhất.

# Save the merged data to a new CSV file
merged_file_path = 'full_data\ocr_data_test\merged_data.csv'
merged_data.to_csv(merged_file_path, index=False)

merged_file_path, merged_data.head()


('full_data\\ocr_data_test\\merged_data.csv',
   images_name                        raw_name                     OCR Result
 0      s1.jpg  Pho Ba Trieu, Pho Hai Ba Trung  PHO HAI BA TRUNG, PHO BATRIEU
 1      s1.jpg  Pho Ba Trieu, Pho Hai Ba Trung  PHO HAI BA TRUNG, PHO BATRIEU
 2      s2.jpg      Pho Hang Giay, Pho Gam Cau     , PHO HANGGIAY, PHO GAACAU
 3      s3.jpg                   Duong De Tham                  DUONG DE THAM
 4      s4.jpg      Thanh, Pho Nguyen Dinh Thi          , @PHO NGUYEN DINHTHI)

In [ ]:
merged_data

,images_name,raw_name,OCR Result
0,s1.jpg,"Pho Ba Trieu, Pho Hai Ba Trung","PHO HAI BA TRUNG, PHO BATRIEU"
1,s1.jpg,"Pho Ba Trieu, Pho Hai Ba Trung","PHO HAI BA TRUNG, PHO BATRIEU"
2,s2.jpg,"Pho Hang Giay, Pho Gam Cau",", PHO HANGGIAY, PHO GAACAU"
3,s3.jpg,Duong De Tham,DUONG DE THAM
4,s4.jpg,"Thanh, Pho Nguyen Dinh Thi",", @PHO NGUYEN DINHTHI"
...,...,...,...
351,s385.jpg,duong tong duy tan,DUONG TONG DUY TAN
352,s386.jpg,duong le van linh,DUONG LE VAN LINH
353,s387.jpg,duong cach mang thang 8,DUONG CACH MANG THANG 8
354,Image Name,NaN,OCR Result


In [ ]:
display(merged_data)

,images_name,raw_name,OCR Result
0,s1.jpg,"Pho Ba Trieu, Pho Hai Ba Trung","PHO HAI BA TRUNG, PHO BATRIEU"
1,s1.jpg,"Pho Ba Trieu, Pho Hai Ba Trung","PHO HAI BA TRUNG, PHO BATRIEU"
2,s2.jpg,"Pho Hang Giay, Pho Gam Cau",", PHO HANGGIAY, PHO GAACAU"
3,s3.jpg,Duong De Tham,DUONG DE THAM
4,s4.jpg,"Thanh, Pho Nguyen Dinh Thi",", @PHO NGUYEN DINHTHI"
5,s5.jpg,"Le Quy Don, Ly Thuong Kiet","LYTRONGKET, LEOTON oad"
6,s6.jpg,"Pho Phung Hung, Pho Phan Dinh Phung","PHUNG HUNG, plate"
7,s10.jpg,"Duong Dai Co Viet, Pho Hue",NaN
8,s10.jpg,"Duong Dai Co Viet, Pho Hue",NaN
9,s11.jpg,Duong Ngo Van Nam,DUONG NGOVAN NAM


# 6. Đánh giá hiệu suất model

Trong trường hợp của bạn, việc đánh giá độ chính xác của mô hình nhận diện chữ Paddle OCR có thể sử dụng một số phương pháp sau:

1. **Độ Chính Xác Tuyệt Đối (Exact Match Accuracy)**: Điều này sẽ đánh giá xem kết quả của OCR có khớp chính xác từng ký tự với tên thực tế hay không. Điều này rất nghiêm ngặt và thường không thích hợp cho OCR nơi mà lỗi nhỏ có thể xảy ra.

2. **Levenshtein Distance (Edit Distance)**: Đây là một phương pháp linh hoạt hơn để đo lường sự khác biệt giữa hai chuỗi. Nó tính toán số lượng chèn, xóa, và thay thế cần thiết để biến đổi một chuỗi thành chuỗi kia.

3. **Word Error Rate (WER)** và **Character Error Rate (CER)**: Đây là các phép đo phổ biến trong đánh giá OCR vì chúng cho thấy tỷ lệ lỗi ở cấp độ từ và ký tự.

4. **F1 Score**: Trong trường hợp có sự cân nhắc giữa việc nhận diện đúng (precision) và không bỏ sót thông tin (recall), F1 Score sẽ là một phép đo tổng hợp tốt.

5. **Manual Checking**: Đối với một số trường hợp, việc kiểm tra thủ công có thể cần thiết để đánh giá chất lượng của những từ hoặc ký tự nhận diện được.

Dựa vào dữ liệu bạn cung cấp, chúng ta có thể sử dụng Levenshtein Distance để đánh giá độ chính xác của từng dự đoán so với giá trị thực tế. Để làm điều này, chúng ta cần chuẩn hóa cả dữ liệu thực tế và dữ liệu OCR để loại bỏ dấu câu, khoảng trắng thừa, và chuyển đổi tất cả các chữ cái thành chữ thường hoặc chữ hoa để đảm bảo tính nhất quán khi so sánh.

Nếu bạn cung cấp cho tôi dữ liệu thô và kết quả OCR như một DataFrame hoặc danh sách các cặp chuỗi, tôi có thể viết một đoạn mã Python để tính toán Levenshtein Distance hoặc bất kỳ phép đo nào khác bạn muốn sử dụng để đánh giá độ chính xác.

In [ ]:
# # Inference from .onnx model
# # Run demo

# # %cd /content/PaddleOCR

# !python ResearchModel/4.Paddleocr_PaddlePaddle/4.1.DataProcessing_AIChallenge2021_PaddleOCR-Vietnamese_hungcao0402/tools/infer/predict_system.py --use_gpu=True --use_onnx=True --drop_score=0.7\
# --det_algorithm="DB" --use_mp=True --total_process_num=2 \
# --image_dir=r'ResearchModel/DATA_TestResearch/crop_sign_test_imgs/cropped_image_0.8504.jpg' \
# --det_model_dir=r'model/model_det.onnx'  \
# --rec_model_dir=r'model/model_rec.onnx' \
# --rec_char_dict_path=r"ResearchModel/4.Paddleocr_PaddlePaddle/4.1.DataProcessing_AIChallenge 2021_PaddleOCR-Vietnamese_hungcao0402/train/general_dict.txt" \
# --draw_img_save_dir='output_img' \
# --vis_font_path=r'D:\OneDrive - codemely/Proj1_NhanDienBienBaoTenDuong_CODEMELY/mecode/ResearchModel/4.Paddleocr_PaddlePaddle/4.1.DataProcessing_AIChallenge2021_PaddleOCR-Vietnamese_hungcao0402/font-times-new-roman.ttf'

usage: predict_system.py [-h] [--use_gpu USE_GPU] [--ir_optim IR_OPTIM]
                         [--use_tensorrt USE_TENSORRT]
                         [--min_subgraph_size MIN_SUBGRAPH_SIZE]
                         [--precision PRECISION] [--gpu_mem GPU_MEM]
                         [--image_dir IMAGE_DIR]
                         [--det_algorithm DET_ALGORITHM]
                         [--det_model_dir DET_MODEL_DIR]
                         [--det_limit_side_len DET_LIMIT_SIDE_LEN]
                         [--det_limit_type DET_LIMIT_TYPE]
                         [--det_db_thresh DET_DB_THRESH]
                         [--det_db_box_thresh DET_DB_BOX_THRESH]
                         [--det_db_unclip_ratio DET_DB_UNCLIP_RATIO]
                         [--max_batch_size MAX_BATCH_SIZE]
                         [--use_dilation USE_DILATION]
                         [--det_db_score_mode DET_DB_SCORE_MODE]
                         [--det_east_score_thresh DET_EAST_SCORE_THRESH]
       